# Introduction
Query auto completion (QAC) is an important component of writing assitance in search engines. Given an incomplete user input, QAC suggests a list of possible completions that help save users' keystrokes. These completions usually start with the user input. For example, when users type in "linke", QAC may suggests "linkedin", "linkedin corporation", etc..

A typical QAC system usually consists of two main components: candidate generation and candidate ranking. Given a user input, the candidate generation component generates a list of candidate suggestions first, and the candidate ranking component ranks the suggestions in the order of relevance. 

In this session, we will go through the steps of building a QAC system. The content covers


1.   sample dataset preparation
2.   candidate generation component construction
3.   candidate ranking component construction
4.   a QAC system in action



# Sample Dataset Preparation
In this section, we will use a small sample of a public dataset, AOL, to generate the queries.

The outputs are:
1.  background queries (used for candidate generation)
2.  training / development / test queries (used for model training and evaluation in candidate ranking)



In [1]:
# Download a sample data of the AOL (America Online) dataset from the web
# The complete collection consists of ~20M web queries collected from ~650k AOL users over three months (2006 March to 2006 June).
!wget http://www.cim.mcgill.ca/~dudek/206/Logs/AOL-user-ct-collection/user-ct-test-collection-01.txt

--2020-08-26 17:43:50--  http://www.cim.mcgill.ca/~dudek/206/Logs/AOL-user-ct-collection/user-ct-test-collection-01.txt
Resolving www.cim.mcgill.ca (www.cim.mcgill.ca)... 132.206.73.26
Connecting to www.cim.mcgill.ca (www.cim.mcgill.ca)|132.206.73.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 222762553 (212M) [text/plain]
Saving to: ‘user-ct-test-collection-01.txt’

user-ct-test-collec 100%[===================>] 212.44M  46.7MB/s    in 4.6s    

2020-08-26 17:43:55 (45.8 MB/s) - ‘user-ct-test-collection-01.txt’ saved [222762553/222762553]



In [2]:
# Take a glimpse into the data. We will only make use of the "Query" and the "QueryTime" column
# TWC: We will only use Query and QueryTime
!head user-ct-test-collection-01.txt

AnonID	Query	QueryTime	ItemRank	ClickURL
142	rentdirect.com	2006-03-01 07:17:12		
142	www.prescriptionfortime.com	2006-03-12 12:31:06		
142	staple.com	2006-03-17 21:19:29		
142	staple.com	2006-03-17 21:19:45		
142	www.newyorklawyersite.com	2006-03-18 08:02:58		
142	www.newyorklawyersite.com	2006-03-18 08:03:09		
142	westchester.gov	2006-03-20 03:55:57	1	http://www.westchestergov.com
142	space.comhttp	2006-03-24 20:51:24		
142	dfdf	2006-03-24 22:23:07		


In [3]:
!wc -l user-ct-test-collection-01.txt

3558412 user-ct-test-collection-01.txt


In [6]:
# Extract query and split background/train/dev/test data based on QueryTime
import string, re, random
random.seed(1234)  # control repeatability

sample_ratio = 0.01

punctuation = string.punctuation
translator = re.compile('[{}]+'.format(re.escape(string.punctuation)))  # regex to identify consecutive punctuations

with open('user-ct-test-collection-01.txt', 'r') as fin:
  bkg_queries = list()
  train_queries = list()
  dev_queries = list()
  test_queries = list()

  for i, line in enumerate(fin):
    if i == 0 or random.uniform(0, 1) > sample_ratio:
      continue
    split_info = line.split('\t')
    
    q = translator.sub(" ", split_info[1].lower().strip()) # replace punctuation with space #TWC: Look into this
    t = split_info[2]

  # To prevent leakage
    if "2006-03-01 00:00:00" <= t < "2006-05-01 00:00:00":
      bkg_queries.append(q) #background query
    elif t < "2006-05-14 00:00:00":
      train_queries.append(q)
    elif t < "2006-05-21 00:00:00":
      dev_queries.append(q) # to prevent le
    elif t < "2006-05-28 00:00:00":
      test_queries.append(q)
    else:
      pass

print("Number of produced queries:")
print("  background queries\t{}".format(len(bkg_queries)))
print("  train queries\t{}".format(len(train_queries)))
print("  dev queries\t{}".format(len(dev_queries)))
print("  test queries\t{}".format(len(test_queries)))

Number of produced queries:
  background queries	23640
  train queries	5057
  dev queries	2431
  test queries	2887


# Candidate Generation Component Construction
In this section, we will use the background queries to construct a finite state transducer (FST) based candidate generator. The process is similar to a trie look up.

Given a query "abc", the FST will look up top k most frequent background queries that starts with "abc".

In [5]:
# Install an open-source autocompletion package that supports trie-based completion (https://zepworks.com/posts/you-autocomplete-me/)
# You can also use ElasticSearch for the same purpose (https://www.elastic.co/guide/en/elasticsearch/reference/current/search-suggesters.html#completion-suggester)
!pip install fast-autocomplete

     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144794 sha256=f6d14500ec24fb23a14569db6e2e9728c816d363df293dab21d9c7e7c3b05bde
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [7]:
# Construct the query logs using background queries
# Write down the logs in a format accepted by the package
from collections import Counter
bkg_query_count = Counter(bkg_queries)

query_log = dict()
for query, count in bkg_query_count.items():
  query_log[query] = [dict(), query, count]

import json
with open('query_log.json', 'w') as outfile:
    json.dump(query_log, outfile, indent=2)

In [8]:
!head -n 10 query_log.json

{
  "staple com": [
    {},
    "staple com",
    1
  ],
  "lottery": [
    {},
    "lottery",
    2


In [9]:
from fast_autocomplete import autocomplete_factory

content_files = {
    'words': {
        'filepath': 'query_log.json',
        'compress': True  # means compress the graph data in memory
    }
}

candidate_generator = autocomplete_factory(content_files=content_files)

In [10]:
candidate_generator.search('goo', max_cost=0, size=5)

[['goo'], ['google'], ['google com'], ['googles'], ['google earth']]

# Candidate Ranking Component Construction
## TWC: Previou, we have candidate generation. We will now use ranking via DL.
In this section, we will go through the steps to use the deep text understanding framework, [DeText](https://github.com/linkedin/detext), to build a CNN ranking model ([CLSM](https://www.microsoft.com/en-us/research/wp-content/uploads/2015/10/spir0468-mitra.pdf)) to rank the queries.

This model uses CNN to extract semantic representations from the prefix and the suffix. We define prefix as the sequence of complete words in the user input, and suffix as the trailing words after the prefix in the completions.

For example, for user input "linkedin sof" and candidate suggestion "linkedin software engineer", the prefix is "linkedin" and the suffix is "software engineer". 

After the representation vectors are obtained, the model then computes the cosine similarity between the two vectors. A learning-to-rank layer is jointly trained.

In [11]:
# Install the DeText package
!pip install detext==2.0.8

     |████████████████████████████████| 71kB 2.1MB/s 
     |████████████████████████████████| 17.4MB 212kB/s 
     |████████████████████████████████| 109.2MB 33kB/s 
     |████████████████████████████████| 40kB 6.3MB/s 
     |████████████████████████████████| 491kB 41.8MB/s 
     |████████████████████████████████| 3.2MB 43.8MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
  Created wheel for detext: filename=detext-2.0.8-cp36-none-any.whl size=77060 sha256=8d11d7f668cb780a3b6981b97a870a113ddc2695684f1b86b85fb132bb51504b
  Stored in directory: /root/.cache/pip/wheels/3d/3e/1e/f093368c4b09221180f4e804ff4d766b9f1015f807613c5520
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=66eee26e692af58220025ad7f84202168961ba7dfeab308013eb9970d76394da
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built detext gast
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have num

In [12]:
%tensorflow_version 1.x
import detext
import tensorflow as tf
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

In [13]:
# Construct vocabularies from background and training queries
vocab_file = 'vocab.txt'
vocab = set(['[PAD]', '[UNK]', '[CLS]', '[SEP]'])
for query in bkg_queries + train_queries:
  vocab.update(query.split(' '))

with open(vocab_file, 'w') as fout:
  for w in vocab:
    fout.write(w + '\n')

In [14]:
#TWC: Convert python to tf_records
def create_bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[val.encode('utf8') for val in value]))

def create_float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def get_prefix(user_input):
  """Returns the prefix in the user input. E.g., 'hello w' -> 'hello'."""
  return ' '.join(user_input.split(' ')[:-1])

def get_suffix(text, prefix):
  """Returns the suffix in the text given prefix. E.g., given text = 'hello world' and prefix = 'hello', suffix_in_words = 'world' """
  return text[len(prefix):].strip()
  
max_candidate_num = 5 #TWC: maximum candidate

def generate_tfrecord(queries, output_file):
  """Generates tfrecord given queries

  For every query, we split them at every positions to simulate user inputs and generate the completions using the candidate generator
  The query before split is the label
  For example, given query "abc", we will generate user input "", "a", "ab", "abc" and the label of the completion "abc" will be 1
  """
  writer = tf.python_io.TFRecordWriter(output_file)
  total_records = 0
  for query in queries:
    if len(query) <= 1:
      continue

    # Split query at every letter to simulate user input
    for i in range(len(query)):
      user_input = query[:i]

      prefix = get_prefix(user_input)

      # Generate completions given user input
      completions = candidate_generator.search(user_input, max_cost=0, size=max_candidate_num)
      
      suggested_suffixes = [get_suffix(completion[0], prefix) for completion in completions]
      labels = [completion[0] == query for completion in completions]

      # Skip cases where no completion is the same as user input
      if sum(labels) == 0:
        continue

      # Compose and write TFRecord
      features = {
          'usr_prefix': create_bytes_feature([prefix]),
          'doc_suffix': create_bytes_feature(suggested_suffixes),
          'label': create_float_feature(labels)
      }
      tf_example = tf.train.Example(features=tf.train.Features(feature=features))
      writer.write(tf_example.SerializeToString())
      
      total_records += 1
  print(f'Converted TFRecord to {output_file}. Number of total record: {total_records}')

generate_tfrecord(train_queries, './train.tfrecord')
generate_tfrecord(dev_queries, './dev.tfrecord')
generate_tfrecord(test_queries, './test.tfrecord')

Converted TFRecord to ./train.tfrecord. Number of total record: 5463
Converted TFRecord to ./dev.tfrecord. Number of total record: 2327
Converted TFRecord to ./test.tfrecord. Number of total record: 2715


In [15]:
from detext.run_detext import run_detext,DetextArg

args = DetextArg(ftr_ext="cnn",
        num_filters=50,
        num_units=64,
        emb_sim_func=["inner"], # cosine matching function
        ltr_loss_fn="softmax", # learning-to-rank loss
        optimizer="bert_adam", # same AdamWeightDecay optimizer as in BERT training
        learning_rate=0.002,
        max_len=16,
        min_len=3,
        use_deep=True,
        num_train_steps=300,
        steps_per_stats=30,
        steps_per_eval=30,
        train_batch_size=64,
        test_batch_size=64,
        pmetric="mrr@5",
        vocab_file=vocab_file,
        feature_names=["label","usr_prefix","doc_suffix"],
        train_file="train.tfrecord",
        dev_file="dev.tfrecord",
        test_file="test.tfrecord",
        out_dir="output")

run_detext(args)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





Unable to check if None is of type typing.Union[typing.List[str], NoneType] for field all_metrics of argument class <class 'detext.run_detext.DetextArg'>
Unable to check if None is of type typing.Union[typing.List[float], NoneType] for field score_rescale of argument class <class 'detext.run_detext.DetextArg'>
Unable to check if None is of type typing.Union[typing.Dict[str, int], NoneType] for field hvd_info of argument class <class 'detext.run_detext.DetextArg'>
Unable to check if None is of type typing.Union[typing.List[int], NoneType] for field task_ids of argument class <class 'detext.run_detext.DetextArg'>
Unable to check if None is of type typing.Union[typing.List[float], NoneType] for field task_weights of argument class <class 'detext.run_detext.DetextArg'>


INFO:tensorflow:Getting executor context info...


INFO:tensorflow:Getting executor context info...


  saving hparams to output/hparams



...

INFO:tensorflow:global_step = 210


INFO:tensorflow:loss = 0.8170104


INFO:tensorflow:loss = 0.8170104


INFO:tensorflow:metric/mrr@5 = 0.79913443


INFO:tensorflow:metric/mrr@5 = 0.79913443


In [ ]:
# Training output of DeText
!ls output

INFO:tensorflow:best_mrr@5				     model.ckpt-210.index
checkpoint				     model.ckpt-210.meta
eval					     model.ckpt-240.data-00000-of-00001
eval_log.txt				     model.ckpt-240.index
events.out.tfevents.1598313201.adb61a8c6e3a  model.ckpt-240.meta
export					     model.ckpt-270.data-00000-of-00001
graph.pbtxt				     model.ckpt-270.index
hparams					     model.ckpt-270.meta
model.ckpt-180.data-00000-of-00001	     model.ckpt-300.data-00000-of-00001
model.ckpt-180.index			     model.ckpt-300.index
model.ckpt-180.meta			     model.ckpt-300.meta
model.ckpt-210.data-00000-of-00001	     network_structure.txt



In [ ]:
# Check model signature
import glob
from tensorflow.contrib import predictor 
saved_model_path = glob.glob('output/export/best_mrr@5/*')[0]

print(saved_model_path)
!saved_model_cli show --dir output/export/best_mrr@5/* --tag_set serve --signature_def serving_default

INFO:tensorflow:output/export/best_mrr@5/1598313254
INFO:tensorflow:The given SavedModel SignatureDef contains the following input(s):
  inputs['doc_suffix'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: doc_suffix_placeholder:0
  inputs['label'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: label_placeholder:0
  inputs['uid'] tensor_info:
      dtype: DT_INT64
      shape: ()
      name: uid_placeholder:0
  inputs['usr_prefix'] tensor_info:
      dtype: DT_STRING
      shape: ()
      name: usr_prefix_placeholder:0
  inputs['weight'] tensor_info:
      dtype: DT_FLOAT
      shape: ()
      name: weight_placeholder:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['label'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, -1)
      name: ExpandDims_2:0
  outputs['scores'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1)
      name: Squeeze:0
  outputs['uid'] tensor_info:
      dtype: DT_INT64
      shape: (1

# A QAC System in Action
In this section, we put the candidate generation and ranking components together into a QAC system.

In [ ]:
# Example of a QAC system
predict_fn = predictor.from_saved_model(saved_model_path)  # Load trained TF model

def get_completions(user_input):
  """Generates completions given user input with scores from neural ranking models"""  
  prefix = get_prefix(user_input)
  completions = candidate_generator.search(user_input, max_cost=0, size=max_candidate_num)
  
  suggested_suffixes = [get_suffix(completion[0], prefix) for completion in completions]

  predictions = predict_fn( 
                  {"doc_suffix": suggested_suffixes,
                   "usr_prefix": prefix,
                   # dummy pass-through input for label field -- this is not used in making predictions
                   "label": [1.0]*len(suggested_suffixes)})
  
  scores = predictions['scores'][0]

  completions_with_scores = [(f"{prefix} {suffix}".strip(), score) for (suffix, score) in zip(suggested_suffixes, scores)]
  completions_with_scores.sort(key=lambda x: (x[1], x[0]), reverse=True)
  return completions_with_scores

user_input = 'fac'
print(get_completions(user_input))

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from output/export/best_mrr@5/1598313254/variables/variables
INFO:tensorflow:[('facebook', 2.0855944), ('facebook com', 1.4921477), ('face painting photos at animal kingdom', -0.0014320329), ('face burn', -0.0014320329), ('face', -0.0014320329)]
